In [2]:
!pip install 'biopandas'
!pip install scipy
!pip install scikit-learn
!pip install biopython

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [3]:
import argparse
import collections
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import subprocess
import time
from biopandas.pdb import PandasPdb
from sklearn.neighbors import KDTree
from collections import defaultdict
import csv
import pandas as pd

In [4]:
data_path = "/home/rnomura/research/analysis/analysis/pdb_files/"
itp_path = "/home/rnomura/research/analysis/analysis/itp_files/"
research_path = "/home/rnomura/research/analysis/analysis/analysis_program/csv/"
pic_path = "/home/rnomura/research/analysis/analysis/analysis_program/pic_stock/"

In [10]:
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP
p = PDBParser()
structure = p.get_structure("minus1", data_path + "minus1.pdb")
model = structure[0]
import warnings

# 警告を無視する
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    # ここにDSSPのコードを書く
    dssp = DSSP(model, data_path + "minus1.pdb", dssp='mkdssp')
    total_asa = sum(residue[3] for residue in dssp)
    print(total_asa)#rsaの和でしかない。

24.886102372778936


In [15]:
dic_max_asa = {"A":129, "R":274, "N":195, "D":193, "C":167, "E":223, "Q":225, "G":104, "H":224, "I":197, "L":201, "K":236, "M":224, "F":240, "P":159, "S":155, "T":172, "W":285, "Y":263, "V":174}

In [16]:
amino_acid = []
rsa        = []
for dssp_data in dssp:
    amino_acid.append(dssp_data[1])
    rsa.append(dssp_data[3])

In [17]:
asa = 0
for amino, rsa_data in zip(amino_acid, rsa):
    asa += dic_max_asa[amino] * float(rsa_data)
asa

4066.426039636177

# ここから自動化する！

In [19]:
import warnings
from Bio.PDB import PDBParser
from Bio.PDB.DSSP import DSSP

In [12]:
dic_HI = {"minus1":[2070], "minus2":[1395, 95, 54, 48, 46], "plus1":[1234, 67, 13], "plus2":[2065], "plus2_ala_A":[3058], "plus2_ala_0":[3391], "sample4":[1061, 76], "sample5":[1156, 93, 65, 47, 41], "sample6":[1168, 78, 42, 16, 9], "sample7":[1607, 48], "sample8":[1573, 98, 23, 14], "sample9":[1616, 49, 16]}

In [13]:
HI_amino = ["A", "C", "F", "I", "L", "M", "V", "W", "Y"]

In [20]:
dic_max_asa = {"A":129, "R":274, "N":195, "D":193, "C":167, "E":223, "Q":225, "G":104, "H":224, "I":197, "L":201, "K":236, "M":224, "F":240, "P":159, "S":155, "T":172, "W":285, "Y":263, "V":174}

In [21]:
THSA = []
RHSA = []
LHP = []
rank_num = []

for keyword in dic_HI.keys():
    p = PDBParser()
    structure = p.get_structure(keyword, data_path + keyword + ".pdb")
    model = structure[0]
    # 警告を無視する
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
    # ここにDSSPのコードを書く
    dssp = DSSP(model, data_path + keyword + ".pdb", dssp='mkdssp')
    HI_amino_acid = []
    HI_rsa        = []
    amino_acid = []
    rsa        = []
    for dssp_data in dssp:
        if dssp_data[1] in HI_amino:
            HI_amino_acid.append(dssp_data[1])
            HI_rsa.append(dssp_data[3])
        amino_acid.append(dssp_data[1])
        rsa.append(dssp_data[3])
    HI_asa = 0
    for amino, rsa_data in zip(HI_amino_acid, HI_rsa):
        HI_asa += dic_max_asa[amino] * float(rsa_data)
    asa = 0
    for amino, rsa_data in zip(amino_acid, rsa):
        asa += dic_max_asa[amino] * float(rsa_data) 
    THSA.append(HI_asa)
    RHSA.append(HI_asa / asa)
    LHP.append(dic_HI[keyword][0])
    rank_num.append(len(dic_HI[keyword]))
print(THSA)
print(RHSA)
print(LHP)

[2480.609552374245, 1799.5178793553664, 1573.0340708075648, 2482.2289720895574, 3647.2924528301883, 4046.462264150943, 1373.0210625998016, 1472.3473740538195, 1547.2936763748132, 1942.9616016507553, 2004.2622983707668, 1954.3413784341597]
[0.6100220508611993, 0.4181303430360963, 0.3618384050593505, 0.574678381333784, 0.8172722662206875, 1.0, 0.3191861475232068, 0.3329823189801138, 0.3473440364092385, 0.46835101689148034, 0.4744255637037935, 0.4795725207038548]
[2070, 1395, 1234, 2065, 3058, 3391, 1061, 1156, 1168, 1607, 1573, 1616]
